In [1]:
# Установка необходимых библиотек
!pip install tensorflow tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import Model

In [3]:
# Загрузка датасета SVHN
(ds_train, ds_test), ds_info = tfds.load(
    'svhn_cropped',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/73257 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.K2WUOW_3.1.0/svhn_cropped-train.tfrecord*...:   0%…

Generating test examples...:   0%|          | 0/26032 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.K2WUOW_3.1.0/svhn_cropped-test.tfrecord*...:   0%|…

Generating extra examples...:   0%|          | 0/531131 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.K2WUOW_3.1.0/svhn_cropped-extra.tfrecord*...:   0%…

Dataset svhn_cropped downloaded and prepared to /root/tensorflow_datasets/svhn_cropped/3.1.0. Subsequent calls will reuse this data.


In [4]:
# Функция предварительной обработки данных
def preprocess(image, label):
    image = tf.image.resize(image, (32, 32))  # Размер изображений для SVHN
    image = image / 255.0  # Нормализация
    return image, label

In [5]:
# Применение предварительной обработки и создание батчей
batch_size = 32
train_ds = ds_train.map(preprocess).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = ds_test.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [6]:
# Определение модели
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.pool1 = MaxPooling2D()
        self.conv2 = Conv2D(64, 3, activation='relu')
        self.pool2 = MaxPooling2D()
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.dropout = Dropout(0.5)
        self.d2 = Dense(10)  # 10 выходных нейронов для 10 классов (цифры от 0 до 9)

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout(x)
        return self.d2(x)

In [7]:
# Создание экземпляра модели
model = MyModel()

In [8]:
# Определение функции потерь и оптимизатора
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


In [9]:
# Метрики
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [10]:
# Функция шага обучения
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [11]:
# Функция шага тестирования
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [12]:
# Обучение модели
EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    # Обучение
    for images, labels in train_ds:
        train_step(images, labels)

    # Тестирование
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.8619576692581177, Accuracy: 73.38327026367188, Test Loss: 0.5375553965568542, Test Accuracy: 85.26380920410156
Epoch 2, Loss: 0.4795370399951935, Accuracy: 86.32221984863281, Test Loss: 0.47767677903175354, Test Accuracy: 86.7176284790039
Epoch 3, Loss: 0.3980330526828766, Accuracy: 88.3834457397461, Test Loss: 0.41800257563591003, Test Accuracy: 88.1236801147461
Epoch 4, Loss: 0.3513958156108856, Accuracy: 89.7809066772461, Test Loss: 0.4133552610874176, Test Accuracy: 88.46495056152344
Epoch 5, Loss: 0.310519814491272, Accuracy: 90.8115234375, Test Loss: 0.4012855887413025, Test Accuracy: 88.97686004638672
